In [3]:
import pytesseract
from pytesseract import Output
import cv2
import re


class OCR_get_total:

    def __init__(self,img_path,keyword='total',seuil=0.5):

        self.img_path = img_path

        self.keyword = keyword

        self.img = cv2.imread(self.img_path)

        self.d = pytesseract.image_to_data(self.img, output_type=Output.DICT)

        self.d['text'] = [s.lower() for s in self.d['text']]

        self.seuil = seuil

        self.n_boxes = len(self.d['text'])

        self.keyword_boxes = {}

        self.dic_lines = {}

        self.result = None


    def get_keyword_boxes(self):

        for i in range(self.n_boxes):

            if self.keyword in self.d['text'][i]:

                self.keyword_boxes[i] = [self.d['left'][i], self.d['top'][i], self.d['width'][i], self.d['height'][i]]



    def get_keyword_lines(self):

        lines = {}

        for key in self.keyword_boxes.keys():

            str_key = ''

            threshold = self.keyword_boxes[key][3]*self.seuil


            for i in range(self.n_boxes):

                if self.d['text'][i] != '':

                    if self.d["left"][i] > (self.keyword_boxes[key][0]+self.keyword_boxes[key][2]):

                        if abs((self.d["top"][i]+ (self.d['height'][i]/2)) - (self.keyword_boxes[key][1]+(self.keyword_boxes[key][3]/2)))<=threshold:

                            str_key += ' ' + self.d['text'][i]

            lines[key] = str_key

        self.dic_lines =  lines



    def get_amount_from_lines(self):

        p = '[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+'
    
        list_of_lines = list(self.dic_lines.values())

        list_of_amounts = []

        for s in list_of_lines:

            if re.search(p, s) is not None:

                for catch in re.finditer(p, s):

                    list_of_amounts.append(catch[0]) 

        list_of_amounts = [s.replace(',','.') for s in list_of_amounts]

        list_of_amounts = [float(s) if s[-1] != '.' else float(s[:-1]) for s in list_of_amounts]

        if list_of_amounts: 

            self.result = max(list_of_amounts)

        else:
            self.result = '?'




    def get_total(self):

        self.get_keyword_boxes()
        self.get_keyword_lines()
        self.get_amount_from_lines()

        if self.result != '?':

            print(f'Total amount for this receipt is : {self.result} $')

        else:

            print('The OCR process failed to detect the total amount')

        


   



           


       

    


    

In [4]:
for k in range(1,40):

    a = OCR_get_total(f'./data/{k}.jpg')
    print( f'receipt  {k} : ')
    a.get_total()

    

receipt  1 : 
Total amount for this receipt is : 125.23 $
receipt  2 : 
Total amount for this receipt is : 15.81 $
receipt  3 : 
Total amount for this receipt is : 82.27 $
receipt  4 : 
Total amount for this receipt is : 41.14 $
receipt  5 : 
Total amount for this receipt is : 49.64 $
receipt  6 : 
Total amount for this receipt is : 44.687 $
receipt  7 : 
Total amount for this receipt is : 13.78 $
receipt  8 : 
Total amount for this receipt is : 30.37 $
receipt  9 : 
Total amount for this receipt is : 52.04 $
receipt  10 : 
Total amount for this receipt is : 30.37 $
receipt  11 : 
Total amount for this receipt is : 12.48 $
receipt  12 : 
The OCR process failed to detect the total amount
receipt  13 : 
Total amount for this receipt is : 30.37 $
receipt  14 : 
Total amount for this receipt is : 27.1 $
receipt  15 : 
Total amount for this receipt is : 35.66 $
receipt  16 : 
Total amount for this receipt is : 107.45 $
receipt  17 : 
Total amount for this receipt is : 17.77 $
receipt  18 : 

In [32]:
a = OCR_get_total('test_2.jpg')

In [34]:
a.get_keyword_boxes()

In [36]:
a.dic_lines

{238: ' 99.04',
 251: ' tax 8.41',
 255: ' 107.45',
 307: ' in',
 320: ' tender 107.45,'}

In [44]:
for k in range(1,25):

    a = OCR_get_total(f'./DDATA/{k}_1.jpg')
    print(k)
    a.get_total()

1
Total amount for this receipt is : 15.81 $
2
Total amount for this receipt is : 1250.0 $
3
Total amount for this receipt is : 49.64 $
4
Total amount for this receipt is : 5.98 $
5
Total amount for this receipt is : 78.0 $
6
Total amount for this receipt is : 52.04 $
7
Total amount for this receipt is : 12.48 $
8
Total amount for this receipt is : 126.96 $
9
Total amount for this receipt is : 27.1 $
10
Total amount for this receipt is : 107.45 $
11
Total amount for this receipt is : 24.01 $
12
Total amount for this receipt is : 45.0 $
13
The OCR process failed to detect the total amount
14
Total amount for this receipt is : 1.09 $
15
The OCR process failed to detect the total amount
16
Total amount for this receipt is : 4.55 $
17
Total amount for this receipt is : 70.0 $
18
Total amount for this receipt is : 5.99 $
19
Total amount for this receipt is : 0.2 $
20
The OCR process failed to detect the total amount
21
Total amount for this receipt is : 26.0 $
22
The OCR process failed to d